In [1]:
import pandas as pd

data = pd.read_csv('Tweets.csv')
data.head()

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [2]:
'''
clean twitter text
'''
import re
import numpy as np

def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

data['text'] = np.array([clean_tweet(tweet) for tweet in data['text']])
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,What said,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,plus you ve added commercials to the experienc...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,I didn t today Must mean I need to take anothe...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,it s really aggressive to blast obnoxious ente...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,and it s a really big bad thing about it,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
# data process
import string
import nltk
from nltk.tokenize import *
from nltk.corpus import stopwords

def cleaned_list_func(tweet):
    new_text = []
    tokens=word_tokenize(tweet)
    for word in tokens:
        lemmatizer = nltk.WordNetLemmatizer()
        new_word = lemmatizer.lemmatize(word)
        if len(new_word) > 3 and new_word not in string.punctuation and new_word.lower() not in stopwords.words('english'):
            new_text.append(new_word.lower())
    return ' '.join(new_text)

data['text'] = data.text.apply(cleaned_list_func)
data.text.head()

0                                                 said
1               plus added commercial experience tacky
2               today must mean need take another trip
3    really aggressive blast obnoxious entertainmen...
4                                         really thing
Name: text, dtype: object

In [4]:
pos_tweets = [ tweet for index, tweet in enumerate(data['text']) if data['airline_sentiment'][index] == 'positive']
neg_tweets = [ tweet for index, tweet in enumerate(data['text']) if data['airline_sentiment'][index] == 'negative']

In [5]:
# LDA model
# LDA for positive tweets
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)

pos_tf = tf_vectorizer.fit_transform(pos_tweets)

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(pos_tf)

d:\python34\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=5, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [7]:
# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         print("Topic #%d:" % topic_idx)
#         print(" ".join([feature_names[i]
#                         for i in topic.argsort()[:-n_top_words - 1:-1]]))
#         print()
#     print()

# n_top_words = 20
# tf_feature_names = tf_vectorizer.get_feature_names()
# print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
plane time like wait weather hour issue told response sitting sure fleek fleet week stuck board delayed leave update making

Topic #1:
flight cancelled flightled delay today thank tomorrow help flighted home email sent getting hotel refund time info booking problems phone

Topic #2:
service customer airline change great going know love worst flying experience thank really work best need better good make look

Topic #3:
thanks gate agent waiting luggage number united airport baggage right line phone time said left passenger tell think tonight care

Topic #4:
flight hour hold late help minute seat need delayed people ticket trying connection check boarding morning flightr miss problem travel




In [7]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, pos_tf, tf_vectorizer)
data = pyLDAvis.sklearn.prepare(lda, pos_tf, tf_vectorizer)
pyLDAvis.display(data)

In [8]:
# LDA for negative tweets
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)

neg_tf = tf_vectorizer.fit_transform(neg_tweets)
lda.fit(neg_tf)
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, neg_tf, tf_vectorizer)
data = pyLDAvis.sklearn.prepare(lda, neg_tf, tf_vectorizer)
pyLDAvis.display(data)

d:\python34\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
